In [ ]:
# MovieDataByYear
import requests as req
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import numpy as np

# 初始年分設定
Set_year = 2022

# dataframe原點 空的
dfsum = pd.DataFrame()

# 要抓到幾年?
while Set_year >= 2017 :
    url = f"https://www.boxofficemojo.com/year/{Set_year}/?ref_=bo_hm_yrdom"
    res = req.get(url)
    soup = bs(res.text, "lxml")
    
    # 取出td裡的元素 => 要用的資料都在table裡
    Movie_DF_By_Years_elements = soup.select("table td")
    
    # 取出抓連結用的元素
    release = soup.find_all("a", href = re.compile("^/release/"))
    
    # td原點 空的
    Data_By_Td = []
    
    # 電影連結原點 空的
    releaselist = []


    for i in Movie_DF_By_Years_elements :
        
        # 有隱藏的td不知道要幹嘛的 排除掉
        if i.get_text()  == "false" or ("hidden" in i.get_attribute_list("class")) :
            continue
        else:
            
            # 需要的資料放list裡
            Data_By_Td.append(i.get_text())
    
    # 做出電影連結        
    for i in release :
        
        # 網站沒提供連結格子紀錄為NaN
        if len(i.get_attribute_list("href")) == 0:
            releaselist.append(np.NaN)
            
        # 網站提供的連結沒有HTTP 拚好後放進list
        else:
            releaselist.append("https://www.boxofficemojo.com/" + i.get_attribute_list("href")[0])
    
    # 電影連結list 轉成 np => 形狀預設 => ? * 1
    link_Column_np = np.array(releaselist).reshape(-1,1)
    
    # RawData_np => 形狀預設 =>? * 7(標題數)
    RawData_np = np.array(Data_By_Td).reshape(-1 , 7)
    
    # 生成年的欄位和np陣列
    yearColumn_np  = np.full([RawData_np.shape[0],1],Set_year)
    
    # np 陣列合併 橫向合併 year 合併 抓下來資料集RawData_np
    np_sum = np.concatenate((yearColumn_np,RawData_np),axis = 1)
    
    # np 陣列合併 橫向合併 上述陣列 合併 電影連結欄位
    np_sum = np.concatenate((np_sum,link_Column_np),axis = 1)
    
    #  命名dataframe的欄位
    DF_Columns = ["year" , "Rank", "Release" , "Gross" , "Theaters" , "Total Gross" , "Release Date" , "Distributor","link"]
    
    # 合併好的陣列放進dataframe
    dftemp = pd.DataFrame(np_sum,columns=DF_Columns)
    
    # 拼接dataframe => 隨年份增加資料 資料量往下增加 ignore_index 重複編號直接往下延續編號
    dfsum = dfsum.append(dftemp,ignore_index=True)
    
    # 一年匯出一份json資料檔
    with open(f"movie{Set_year}.json", "w", encoding='utf-8') as file:
        file.write(dfsum[dfsum['year'] == f"{Set_year}"].to_json(orient="columns"))
    
    # 做完一圈年分減一
    Set_year -= 1
    